In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
BMJ_data_all__b_new = pd.read_csv('../input/BMJ-data-all--b-new.csv', sep='\t')
item_profiles1 = pd.read_csv('../input/item-profiles1.csv', sep=';')
item_profiles2 = pd.read_csv('../input/item-profiles2.csv', sep=';')
item_profiles3 = pd.read_csv('../input/item-profiles3.csv', sep=';')
ratings = pd.read_csv('../input/user-item-rating.csv', sep='\t', names=['user_id','item_id','rating'])

In [3]:


def content_based_predict(train, test):
    
    train_ratings = ratings#.iloc[train]
    
    recipe_id_ingredient_id = item_profiles3[['Recipe ID', 'Ingredient ID']]
    ingredientRatings = train_ratings.merge(recipe_id_ingredient_id, left_on='item_id', right_on='Recipe ID')
    ingredientRatings = ingredientRatings[['user_id', 'item_id', 'Ingredient ID', 'rating']]
    ingredientRatings = ingredientRatings.rename(columns={'Ingredient ID': 'ingredient_id', 'item_id': 'recipe_id'})

    
    recipes_dict = defaultdict(list) 
    ingredients_dict = defaultdict(lambda: defaultdict(list))  
    for user_id, recipe_id, ingredient_id, rating in ingredientRatings.itertuples(name='rating', index=False):
        recipes_dict[recipe_id].append(ingredient_id) 
        ingredients_dict[user_id][ingredient_id].append(rating)
 
    for key in ingredients_dict.copy().keys():
        for value in ingredients_dict[key].copy().keys():
            ingredients_dict[key][value]  = sum(ingredients_dict[key][value])/len(ingredients_dict[key][value])
    
    # predict all recepies
   # users = test_ratings['user_id'].drop_duplicates().to_frame()
   # recipes = test_ratings['item_id'].drop_duplicates().to_frame().rename(columns={'item_id': 'recipe_id'})
   # users['key'] = 1
   # recipes['key'] = 1
   # users_recipes = users.merge(recipes, how='outer').drop("key", 1)
   # users_recipes['rating_est'] = np.nan
    
    test_ratings = ratings#.iloc[test].rename(columns={'item_id': 'recipe_id'})
    
    content_based_predictions = defaultdict(lambda: defaultdict())                        
    for user_id, recipe_id, _ in test_ratings.itertuples(name='predictions', index=False):
        rating_est = 0.0
        counter = 0
        for ingredient_id in recipes_dict[recipe_id]:
            if(type(ingredients_dict[user_id][ingredient_id]) == float):
                rating_est += ingredients_dict[user_id][ingredient_id]
                counter += 1
        if(counter != 0):
            content_based_predictions[user_id][recipe_id] = rating_est/counter
        else:
            content_based_predictions[user_id][recipe_id] = np.nan

    test_ratings['rating_est'] = test_ratings.apply(lambda x: content_based_predictions[x.user_id][x.recipe_id], axis=1)
    test_ratings['details'] = "{'is it awesome': Yes}"
    test_ratings.columns = ['uid', 'iid', 'r_ui', 'est', 'details']
    predictions = list(test_ratings.itertuples(name='Prediction', index=False))
    
    return predictions

In [4]:
# We use this healthiness method to calculate the healthiness of recipes, and to set weights for the post filtering.

In [5]:
def healthiness(itemsDataframe):
    
    # Calculate the energy percentage of each relevant macronutrient. 1g of fat contains 9 kCal. 
    fat = ((itemsDataframe['Fat (g)'] * 9) / itemsDataframe['Calories (kCal)']) * 100
    sugar = ((itemsDataframe['Sugar (g)'] * 4) / itemsDataframe['Calories (kCal)']) * 100
    saturatedFat = ((itemsDataframe['Saturated Fat (g)'] * 9) / itemsDataframe['Calories (kCal)']) * 100
    
    # This calculates a continous version of the healthiness score. Fat/3 because the recommended limit for fat is 3X the others.
    itemsDataframe['Unhealtiness'] = (fat / 3) + sugar + saturatedFat
    
    # These are bounderies and points are tunable to influence 
    # post filter weights in accordence with the health recommendations.
    itemsDataframe.loc[fat > 30, 'fatPoints'] = 0
    itemsDataframe.loc[fat >= 40, 'fatPoints'] = -1
    itemsDataframe.loc[fat >= 50, 'fatPoints'] = -2
    itemsDataframe.loc[fat >= 60, 'fatPoints'] = -3
    itemsDataframe.loc[fat >= 70, 'fatPoints'] = -4
    itemsDataframe.loc[fat >= 80, 'fatPoints'] = -5
    itemsDataframe.loc[fat <= 30, 'fatPoints'] = 1 
    itemsDataframe.loc[fat <= 20, 'fatPoints'] = 2
    itemsDataframe.loc[fat <= 10, 'fatPoints'] = 3
    itemsDataframe.loc[fat <= 5, 'fatPoints'] = 4 
    itemsDataframe.loc[fat <= 1, 'fatPoints'] = 5

    itemsDataframe.loc[sugar > 10, 'sugarPoints'] = 0
    itemsDataframe.loc[sugar >= 13, 'sugarPoints'] = -1
    itemsDataframe.loc[sugar >= 16, 'sugarPoints'] = -2
    itemsDataframe.loc[sugar >= 19, 'sugarPoints'] = -3
    itemsDataframe.loc[sugar >= 22, 'sugarPoints'] = -4
    itemsDataframe.loc[sugar >= 25, 'sugarPoints'] = -5
    itemsDataframe.loc[sugar <= 10, 'sugarPoints'] = 1 
    itemsDataframe.loc[sugar <= 7, 'sugarPoints'] = 2
    itemsDataframe.loc[sugar <= 5, 'sugarPoints'] = 3 
    itemsDataframe.loc[sugar <= 3, 'sugarPoints'] = 4
    itemsDataframe.loc[sugar <= 1, 'sugarPoints'] = 5 
        
    itemsDataframe.loc[saturatedFat > 10, 'satFatPoints'] = 0
    itemsDataframe.loc[saturatedFat >= 13, 'satFatPoints'] = -1
    itemsDataframe.loc[saturatedFat >= 16, 'satFatPoints'] = -2
    itemsDataframe.loc[saturatedFat >= 19, 'satFatPoints'] = -3
    itemsDataframe.loc[saturatedFat >= 22, 'satFatPoints'] = -4
    itemsDataframe.loc[saturatedFat >= 25, 'satFatPoints'] = -5
    itemsDataframe.loc[saturatedFat <= 10, 'satFatPoints'] = 1 
    itemsDataframe.loc[saturatedFat <= 7, 'satFatPoints'] = 2
    itemsDataframe.loc[saturatedFat <= 5, 'satFatPoints'] = 3 
    itemsDataframe.loc[saturatedFat <= 3, 'satFatPoints'] = 4
    itemsDataframe.loc[saturatedFat <= 1, 'satFatPoints'] = 5 
        
       
    itemsDataframe['Healthiness'] = itemsDataframe['fatPoints'] + itemsDataframe['satFatPoints'] + itemsDataframe['sugarPoints']

In [6]:
# Apply healthiness evaluation to each recipe

In [7]:
healthiness(item_profiles2)

In [8]:
from collections import defaultdict
from sklearn.metrics import ndcg_score

def evaluations_at_k(predictions):

    # K is the number of highest ranking predictions to consider
    k = 10
    # This is the threshold for what is considered an adequate recommendation.
    threshold = 3.5
    
    # We map the predictions to the users. uid=user identity, iid=item identity, true_r[ating], est[imated rating]. 
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Sort the predictions for each user and retrieve the k highest ones and put them in a dictionary.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:k]
        
    # Remove the dictionary entries that didn't have enough ratings because of the data partitioning. 
    top_k = top_n.copy()
    for uid, user_ratings in top_k.items():
        if (len(top_k[uid]) != 10):
            top_n.pop(uid)
            
    # Calculate the healthiness average of all recommendations. 
    # 0 = the tipping point between inside and outside of HDR recommendations. Negative value means unhealthy.
    healthinessAverage = 0
    count = 0
    for x in top_n.items():
        count =  count + 1
        healthiness = 0
        for y in x[1]:
            healthiness = healthiness + item_profiles2.at[y[0], 'Healthiness']
        healthinessAverage = healthinessAverage + (healthiness/10)
    
    healthinessAverage = healthinessAverage/count
    
    # This time we put put the estimated rating and the true rating in the dictionary.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        
    # Extract relevant lists from the named tuples in predictions, and calculate NDCG.
    predictions_list = [[x[3] for x in predictions]]
    true_scores = [[x[2] for x in predictions]]
    ndcg = ndcg_score(true_scores, predictions_list, k)
    
    # Calculate the averages of precision and recall
    precision = (sum(prec for prec in precisions.values())) / len(precisions)
    recall = (sum(rec for rec in recalls.values())) / len(recalls)
    
    
    return precision, recall, ndcg, healthinessAverage
    

In [9]:
# This is the post filter method. Suprise predictions are given in a list of immutable tuples.
# To post filter the predictions we make a dataframe of the predictions list, change the predictions,
# and make a new list of tuples.

In [10]:
ratings[['item_id', 'user_id', 'rating']]

,item_id,user_id,rating
0,50,455,3.0
1,457,455,4.0
2,28,455,5.0
3,458,455,3.0
4,459,455,5.0
...,...,...,...
50676,131,84839,3.0
50677,109,84839,5.0
50678,145,84839,5.0
50679,133,84839,5.0


In [11]:
def postfilter(predictions, healthinessFactor):
    predictionsDF = pd.DataFrame.from_records(predictions, columns=['uid', 'iid', 'r_ui', 'est', 'details'])
    ratingsProcessed = predictionsDF.copy()
    ratingsProcessed = ratingsProcessed.join(item_profiles2['Healthiness'], how= 'inner', on= 'iid', sort=False)
    # This is were the values are transformed based on the healthinessFactor constructor and the health points of the recipes.
    ratingsProcessed['est'] = ratingsProcessed['est'] + (ratingsProcessed['Healthiness']*healthinessFactor)
    ratingsProcessed.drop(labels='Healthiness', axis=1, inplace=True)
    ratingsProcessed = list(ratingsProcessed.itertuples(name='Prediction', index=False))
    return ratingsProcessed

In [12]:
# We run the algoritm again, but this time we use post filtering and automatic tuning.

In [13]:
from sklearn.model_selection import KFold

# We use 5 fold cross evaluation.
kf = KFold(n_splits=5)

# Run the algorithm 5 times and evaluate.
for train, test in kf.split(ratings):
    predictions1 = content_based_predict(train, test)
    precision, recall, ndcg, healthinessAverage = evaluations_at_k(predictions1)
   
    print("Precision:", precision, "Recall:", recall, "NDCG_score:", ndcg, "Healthiness:", healthinessAverage)

AttributeError: 'Series' object has no attribute 'recipe_id'

In [ ]:
predictions1

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

reader = Reader()
data = Dataset.load_from_df(ratings, reader)
# there is a trade off between NDCG score and recall. Tuning the parameters may increase one and lower the other, or lower both..
param_grid = {'n_epochs': [5,5], 'lr_all': [0.01, 0.1], 'reg_all':[0.01,0.1]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
grid_search.fit(data)
print(grid_search.best_params['rmse'])
svd = grid_search.best_estimator['rmse']

In [ ]:
kf = KFold(n_splits=5)

precisionAvg = 0
recallAvg = 0
ndcgAvg = 0
healthinessAvg = 0
for trainset, testset in kf.split(data):
    svd.fit(trainset)
    # The second parameter is the post fileter healthiness factor.
    predictions2 = postfilter(svd.test(testset), 0.1)
    precision, recall, ndcg, healthinessAverage = evaluations_at_k(predictions2)
    precisionAvg = precisionAvg + precision
    recallAvg = recallAvg + recall
    ndcgAvg = ndcgAvg + ndcg
    healthinessAvg = healthinessAvg + healthinessAverage
    
print("Precision:", precisionAvg/5, "Recall:", recallAvg/5, "NDCG_score:", ndcgAvg/5, "Healthiness:", healthinessAvg/5)

In [ ]:
predictions2

In [ ]:
#Precision: 0.8732584188405793 Recall: 0.9377091822972897 NDCG_score: 0.9463405507526241 Healthiness: -0.03859170970233361